In [12]:
import re
import string
import pandas as pd
from functools import reduce
from math import log

## Simple example of [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)
1. Example of corpus
2. Preprocessing and Tokenizing
3. Calculating bag of words
4. TF
5. IDF
6. TF-IDF

In [13]:
#1
corpus = """
Armstrong Calculus
Arts Integration in Elementary Curriculum: 2nd Edition
Becoming America: An Exploration of American Literature from Precolonial to Post-Revolution
""".split("\n")[1:-1]

In [14]:
#2
l_A = corpus[0].lower().split()
l_B = corpus[1].lower().split()
l_C = corpus[2].lower().split()

print(l_A)
print(l_B)
print(l_C)

['armstrong', 'calculus']
['arts', 'integration', 'in', 'elementary', 'curriculum:', '2nd', 'edition']
['becoming', 'america:', 'an', 'exploration', 'of', 'american', 'literature', 'from', 'precolonial', 'to', 'post-revolution']


In [15]:
#3
word_set = set(l_A).union(set(l_B)).union(set(l_C))
print(word_set)

{'to', 'in', 'literature', 'precolonial', 'arts', 'elementary', 'calculus', 'integration', 'becoming', 'curriculum:', 'edition', 'from', 'exploration', 'american', 'america:', 'post-revolution', 'armstrong', 'an', 'of', '2nd'}


In [16]:
word_dict_A = dict.fromkeys(word_set, 0)
word_dict_B = dict.fromkeys(word_set, 0)
word_dict_C = dict.fromkeys(word_set, 0)

for word in l_A:
    word_dict_A[word] += 1

for word in l_B:
    word_dict_B[word] += 1

for word in l_C:
    word_dict_C[word] += 1

pd.DataFrame([word_dict_A, word_dict_B, word_dict_C])

,to,in,literature,precolonial,arts,elementary,calculus,integration,becoming,curriculum:,edition,from,exploration,american,america:,post-revolution,armstrong,an,of,2nd
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,1,0,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,1
2,1,0,1,1,0,0,0,0,1,0,0,1,1,1,1,1,0,1,1,0


## \#4 tf - term frequency
In the case of the term frequency $tf(t,d)$, the simplest choice is to use the raw count of a term in a string. 
$${\displaystyle \mathrm {tf} (t,d)={\frac {n_{t}}{\sum _{k}n_{k}}}} $$
where $n_t$ is the number of occurrences of the word $t$ in the string, and in the denominator - the total number of words in this string.

In [17]:
def compute_tf(word_dict, l):
    tf = {}
    sum_nk = len(l)
    for word, count in word_dict.items():
        tf[word] = count/sum_nk
    return tf

In [18]:
#Frecuencia de termino (tf)
tf_A = compute_tf(word_dict_A, l_A)
tf_B = compute_tf(word_dict_B, l_B)
tf_C = compute_tf(word_dict_C, l_C)

# \#5 idf - inverse document frequency
idf is a measure of how much information the word provides
$$ \mathrm{idf}(t, D) =  \log \frac{N}{|\{d \in D: t \in d\}|} $$
- $N$: total number of strings in the corpus ${\displaystyle N={|D|}}$
- ${\displaystyle |\{d\in D:t\in d\}|}$  : number of strings where the term ${\displaystyle t}$ appears (i.e., ${\displaystyle \mathrm {tf} (t,d)\neq 0})$. If the term is not in the corpus, this will lead to a division-by-zero. It is therefore common to adjust the denominator to ${\displaystyle 1+|\{d\in D:t\in d\}|}$.

In [19]:
def compute_idf(strings_list):
    n = len(strings_list)
    idf = dict.fromkeys(strings_list[0].keys(), 0)
    for l in strings_list:
        for word, count in l.items():
            if count > 0:
                idf[word] += 1
    
    for word, v in idf.items():
        idf[word] = log(n / float(v))
    return idf

In [20]:
#frecuencia de documento Inversa
idf = compute_idf([word_dict_A, word_dict_B, word_dict_C])

## \# 6 tf-idf
Then tf–idf is calculated as
$$ {\displaystyle \mathrm {tfidf} (t,d,D)=\mathrm {tf} (t,d)\cdot \mathrm {idf} (t,D)} $$

In [21]:
def compute_tf_idf(tf, idf):
    tf_idf = dict.fromkeys(tf.keys(), 0)
    for word, v in tf.items():
        tf_idf[word] = v * idf[word]
    return tf_idf

In [22]:
tf_idf_A = compute_tf_idf(tf_A, idf)
tf_idf_B = compute_tf_idf(tf_B, idf)
tf_idf_C = compute_tf_idf(tf_C, idf)

In [23]:
dataTf_Idf = pd.DataFrame([tf_idf_A, tf_idf_B, tf_idf_C])
dataTf_Idf
#dataTf_Idf.dtypes
#to print heads
#dataTf_Idf.columns.values

,to,in,literature,precolonial,arts,elementary,calculus,integration,becoming,curriculum:,edition,from,exploration,american,america:,post-revolution,armstrong,an,of,2nd
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.549306,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.549306,0.000000,0.000000,0.000000
1,0.000000,0.156945,0.000000,0.000000,0.156945,0.156945,0.000000,0.156945,0.000000,0.156945,0.156945,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.156945
2,0.099874,0.000000,0.099874,0.099874,0.000000,0.000000,0.000000,0.000000,0.099874,0.000000,0.000000,0.099874,0.099874,0.099874,0.099874,0.099874,0.000000,0.099874,0.099874,0.000000


# For clustering we must use tf-idf weights
the example above is just an example, in practice it is better to apply [TfidfVectorizer from sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [25]:
myKmeans = KMeans(n_clusters=2).fit(dataTf_Idf)
centroids = myKmeans.cluster_centers_
print(centroids)

[[0.         0.         0.         0.         0.         0.
  0.54930614 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.54930614 0.
  0.         0.        ]
 [0.04993692 0.07847231 0.04993692 0.04993692 0.07847231 0.07847231
  0.         0.07847231 0.04993692 0.07847231 0.07847231 0.04993692
  0.04993692 0.04993692 0.04993692 0.04993692 0.         0.04993692
  0.04993692 0.07847231]]


In [26]:
plt.scatter(dataTf_Idf['cheese'],dataTf_Idf['with'],dataTf_Idf['simple'],dataTf_Idf['cats'],dataTf_Idf['another'], c= myKmeans.labels_.astype(float), s=50, alpha=0.5)
plt.scatter(centroids[:, 0], centroids[:, 1], c='red', s=50)

KeyError: 'cheese'

## Full text for clusterring

This corpus contain some strings about Google and some strings about TF-IDF from Wikipedia. Just for example

In [14]:
all_text = """
Google and Facebook are strangling the free press to death. Democracy is the loser
Your 60-second guide to security stuff Google touted today at Next '18
A Guide to Using Android Without Selling Your Soul to Google
Review: Lenovo’s Google Smart Display is pretty and intelligent
Google Maps user spots mysterious object submerged off the coast of Greece - and no-one knows what it is
Android is better than IOS
In information retrieval, tf–idf or TFIDF, short for term frequency–inverse document frequency
is a numerical statistic that is intended to reflect
how important a word is to a document in a collection or corpus.
It is often used as a weighting factor in searches of information retrieval
text mining, and user modeling. The tf-idf value increases proportionally
to the number of times a word appears in the document
and is offset by the frequency of the word in the corpus
""".split("\n")[1:-1]

## Preprocessing and tokenizing
Firstly, we must bring every chars to lowercase and remove all punctuation, because it's not important for our task, but is very harmful for clustering algorithm. 
After that, we'll split strings to array of words.

In [15]:
def preprocessing(line):
    line = line.lower()
    line = re.sub(r"[{}]".format(string.punctuation), " ", line)
    return line

Now, let's calculate tf-idf for this corpus

In [16]:
tfidf_vectorizer = TfidfVectorizer(preprocessor=preprocessing)
tfidf = tfidf_vectorizer.fit_transform(all_text)

And train simple kmeans model with k = 2

In [17]:
kmeans = KMeans(n_clusters=2).fit(tfidf)

Predictions

In [18]:
lines_for_predicting = ["tf and idf is awesome!", "some androids is there"]
kmeans.predict(tfidf_vectorizer.transform(lines_for_predicting))

array([1, 1], dtype=int32)